In [114]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import utils_functions as uf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [13]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

x_tr, y_tr, x_te, y_te = uf.split_data(tX, y, 0.8, 42)

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [7]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

OSError: ../data/test.csv not found.

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

# Basic model
In order to have a basis for comapraison let's first train the unprocessed data with a simple least sqaures regression and then try out other models

In [43]:
np.set_printoptions(suppress=True)

## Least Square (LS)

In [44]:
uf.least_square_training(x_tr, y_tr, x_te, y_te)

Training RMSE: 1.1653644902783922 Testing RMSE: 1.1669195664227903 with accuarcy: 0.74364


array([ 0.00007744, -0.00722011, -0.00597537, -0.00059088, -0.01612306,
        0.00045888, -0.0264537 ,  0.32153919,  0.00002127, -1.08608332,
       -0.22117096,  0.09491042,  0.05946626,  1.094327  , -0.00031902,
       -0.00077106,  1.09905479, -0.00077953,  0.00213603,  0.00365602,
        0.0009131 , -0.00054035, -0.33255812, -0.00129591,  0.00040048,
        0.00134428, -0.00144189, -0.00514942, -0.01030838,  1.08648088])

## Least Square with NaN set to median

In [99]:
medians_x_tr, medians_x_te = uf.set_NaN_to_median(x_tr, x_te)

uf.least_square_training(medians_x_tr, y_tr, medians_x_te, y_te)

Training RMSE: 1.1720182004997715 Testing RMSE: 1.1722323451648153 with accuarcy: 0.74174


array([ 0.00012398, -0.00729644, -0.00548515,  0.00031093, -0.0495109 ,
        0.00053186, -0.03220468,  0.29337725, -0.00117662, -1.13223555,
       -0.24531175,  0.11229847,  0.09388955,  1.13906161, -0.00243037,
        0.002213  ,  1.14502051, -0.00247479,  0.00625913,  0.00257555,
        0.00426097, -0.00055688, -0.21889875, -0.00327839,  0.01708386,
        0.02137633, -0.00532617, -0.00033221, -0.00655618,  1.13444507])

## Least Square with NaN set to mean

In [97]:
means = np.mean(x_tr, axis=0)
means_x_tr = np.copy(x_tr)

for col_idx, mean in enumerate(means):
    idx = means_x_tr[:, col_idx] == -999
    means_x_tr[idx, col_idx] = mean

means = np.mean(x_te, axis=0)
means_x_te = np.copy(x_te)

for col_idx, mean in enumerate(means):
    idx = means_x_te[:, col_idx] == -999
    means_x_te[idx, col_idx] = mean

uf.least_square_training(means_x_tr, y_tr, means_x_te, y_te)

Training RMSE: 1.1654224906508255 Testing RMSE: 1.1669448992148714 with accuarcy: 0.74356


array([ 0.00040143, -0.00724396, -0.00635298, -0.00047231, -0.01682224,
        0.00046002, -0.0265313 ,  0.3250409 ,  0.00005356, -1.08575011,
       -0.22334838,  0.09295235,  0.06037157,  1.09391865, -0.00035183,
       -0.00072761,  1.09878013, -0.00083842,  0.00221064,  0.00339779,
        0.00094102, -0.00053157, -0.3378999 , -0.00133256,  0.00067821,
        0.00162475, -0.00149625, -0.00506126, -0.01023874,  1.08617962])

## Least square with zscore normalized data

In [123]:
norm_x_tr, norm_x_te = uf.zscore_normalize(x_tr, x_te)
uf.least_square_training(norm_x_tr, y_tr, norm_x_te, y_te)

Training RMSE: 1.247432223541836 Testing RMSE: 1.4978955391074478 with accuarcy: 0.6424


array([   0.02811954,   -0.25292855,   -0.25257918,   -0.03302787,
          0.14000493,    0.28665547,  -10.95275325,    0.26545622,
         -0.00108976, -144.30749362,   -0.18269825,    0.11386122,
         18.37380885,   28.04949182,   -0.00034255,   -0.00146194,
         27.75117944,   -0.00072154,    0.00365469,    0.12093259,
          0.00140685,   -0.0627815 ,   -0.20688222,   -0.04899194,
         -0.12036083,    0.32592446,    0.09260865,   -2.68055139,
         -5.02897346,  122.26104408])

## Least square with min-max normalized data

In [124]:
norm_x_tr = np.copy(x_tr)

max_x_tr = np.reshape(np.max(norm_x_tr, axis=0), (1,-1))
min_x_tr = np.reshape(np.min(norm_x_tr, axis=0), (1,-1))

norm_x_tr = (norm_x_tr - min_x_tr)/(max_x_tr - min_x_tr)

norm_x_te = np.copy(x_te)

max_x_te = np.reshape(np.max(norm_x_te, axis=0), (1,-1))
min_x_te = np.reshape(np.min(norm_x_te, axis=0), (1,-1))

norm_x_te = (norm_x_te - min_x_te)/(max_x_te - min_x_te)
uf.least_square_training(norm_x_tr, y_tr, norm_x_te, y_te)

Training RMSE: 1.1649474603667394 Testing RMSE: 182.74668081688333 with accuarcy: 0.34442


array([     0.13714974,     -4.26954082,     -8.15882217,     -1.4697666 ,
            0.28874867,      2.60068882,    -24.5438138 ,      1.84557509,
           -0.13696162,  11104.1292798 ,     -4.2630246 ,      0.26968721,
           40.5076869 ,  -3700.78229188,     -0.00155269,     -0.0052036 ,
        -2673.89405513,     -0.00290359,      0.01244937,     10.39126518,
            0.00477867,     -0.98965558,     -0.63537787,     -0.1968949 ,
           -0.23395344,      0.65566874,      0.32793175,     -5.92486002,
          -11.11477783, -10042.47453384])

## Least square with zscoore and NaN set to median

In [101]:
medians_x_tr, medians_x_te = uf.set_NaN_to_median(x_tr, x_te)
norm_x_tr, norm_x_te = uf.zscore_normalize(medians_x_tr, medians_x_te)
uf.least_square_training(norm_x_tr, y_tr, norm_x_te, y_te)

Training RMSE: 1.2352147611349524 Testing RMSE: 1.2370720257249155 with accuarcy: 0.7063


array([  1.67013792,   1.02084249,   0.97694865,   1.98520393,
       -15.48298645,   1.8875292 ,   1.7826972 ,  41.5398057 ,
         1.59722807,   4.9938043 , -24.89926151,  11.4716026 ,
        19.45704165,  -0.71833692,   1.36138495,   2.03567774,
        -0.22842337,   1.09117482,   2.42605829,   1.80366837,
         2.20047894,   1.55578361, -19.38861231,   1.15705503,
         3.06692096,   3.13969722,   0.95136099,   1.55585583,
         1.78770187,  -1.4850777 ])

## Ridge Regression